# Magnetic Field Interpolator

In [13]:
import numpy as np
import magpylib as magpy
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os

## Generating the Fields Samples using Magpylib

### 1 - Generating the MagField along x-axis

In [6]:
def random_mag_field(d_range, n_magnets_range, sample_direction, N_pts):
    n_magnets = np.random.randint(n_magnets_range[0], n_magnets_range[1]+1)
    sources = []
    B_proj = []

    for i in range(n_magnets):
        # random magnet dimension (in mm)
        dim = np.random.uniform(3, 15, size=3)

        # random magnetization direction (unit vector)
        m_dir = np.random.normal(size=3)
        m_dir = m_dir / np.linalg.norm(m_dir)
        m_mag = np.random.uniform(0.2, 1.2)
        magnetization = m_mag * m_dir

        # create magnet
        cube = magpy.magnet.Cuboid(dimension = dim, 
                                   magnetization = magnetization)
        
        # random position        
        pos = np.random.uniform(-20, 20, size=3)
        cube.position = tuple(pos)

        # random orientation
        euler = np.random.uniform(0, 360, size=3)
        cube.rotate_from_euler(euler, "xyz")

        sources.append(cube)
    
    magnetSet = magpy.Collection(sources)

    # create the direction for the sample
    d = np.array(sample_direction)
    d = d / np.linalg.norm(d)

    t_values = np.linspace(d_range[0], d_range[1], N_pts)
    for t in t_values:
        B_value = magnetSet.getB(t*d)
        B_proj_scalar = np.dot(B_value, d)
        B_proj.append(B_proj_scalar)
    return B_proj


### 2 - Function to Sample Random Points

In [7]:
def sample_points(N_sample, B_field):
        N = len(B_field)
        B_sample = []
        indices = np.random.choice(N, N_sample, replace=False)
        print(indices)
        for idx in indices:
            B_sample.append(B_field[idx])
        return B_sample
            

## Creating the Dataset

In [8]:
"""for i in range(500):
    B_field = random_mag_field((-20, 20), (1,8), (1,0,0), 500)
    x = np.linspace(-20, 20, 500)
    filename = f"./data/sample_{i:03d}.npz"
    np.savez(filename, B_field=B_field, x=x)"""

'for i in range(500):\n    B_field = random_mag_field((-20, 20), (1,8), (1,0,0), 500)\n    x = np.linspace(-20, 20, 500)\n    filename = f"./data/sample_{i:03d}.npz"\n    np.savez(filename, B_field=B_field, x=x)'

## Preprocessing of the Data

### 1 - Dataset

In [12]:
class mag_data(Dataset):
    def __init__(self, data_folder, N_samples = 100):
        self.data_folder = data_folder
        self.files = sorted([
            os.path.join(data_folder, f)
            for f in os.listdir(data_folder)
            if f.endswith(".npz")
        ])
        self.N_samples = N_samples
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):

        data = np.load(self.files[idx])
        B_field = data["B_field"]
        x = data["x"]
        N = len(x)

        idxs = np.random.choice(N, size=self.N_samples, replace=False)

        x_sub = x[idxs]
        B_sub = B_field[idxs]

        sort_idx = np.argsort(x_sub)
        x_sub = x_sub[sort_idx]
        B_sub = B_sub[sort_idx]

        x_sub = torch.tensor(x_sub, dtype=torch.float32).unsqueeze(-1)  # (N_samples, 1)
        B_sub = torch.tensor(B_sub, dtype=torch.float32)

        return x_sub, B_sub


### 2 - DataLoader

In [15]:
dataset = mag_data("./data", 100)

dataloader = DataLoader(
    dataset, 
    batch_size = 8, 
    shuffle = True, 
    num_workers = 0
)


## Neural Network Architecture (SIREN)